In [ ]:
import timm

In [ ]:
from pprint import pprint

pprint(timm.list_models('yolo'))

In [ ]:
m = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
m.eval()

In [ ]:
from PIL import Image

im = Image.open('/Users/ke/Documents/Cal/research/physical-reasoning/demo/demo.jpg')
im

In [ ]:
m.pretrained_cfg

In [ ]:
data_cfg = timm.data.resolve_data_config(m.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg)
transform

In [ ]:
import torch


image_tensor = transform(im)
image_as_batch = image_tensor.unsqueeze(0)

output = m(image_as_batch)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

values, indices = torch.topk(probabilities, 5)

In [ ]:
import requests


IMAGENET_1k_URL = 'https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt'
IMAGENET_1k_LABELS = requests.get(IMAGENET_1k_URL).text.strip().split('\n')
[{'label': IMAGENET_1k_LABELS[idx], 'value': val.item()} for val, idx in zip(values, indices)]

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
from transformers import RTDetrForObjectDetection, RTDetrImageProcessor
from PIL import Image, ImageDraw
import requests
import torch

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
image = im.copy()

image_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-101")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-101")
# image_processor = RTDetrImageProcessor.from_pretrained("PekingU/rtdetr_r50vd")
# model = RTDetrForObjectDetection.from_pretrained("PekingU/rtdetr_r50vd")
# RTDetrForObjectDetection - PekingU/rtdetr_r101vd is even worse than PekingU/rtdetr_r101vd_coco_o365
# 

# prepare image for the model
inputs = image_processor(images=image, return_tensors="pt")

# forward pass
outputs = model(**inputs)

logits = outputs.logits
list(logits.shape)

boxes = outputs.pred_boxes
list(boxes.shape)

# convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[
    0
]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

# show the image with bounding boxes using ImageDraw

image_draw = ImageDraw.Draw(image)
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    image_draw.rectangle(box, outline="red", width=2)
    image_draw.text(box, f"{model.config.id2label[label.item()]} {round(score.item(), 3)}")

image
